In [ ]:
from sqlalchemy import or_
from sqlalchemy.orm import sessionmaker
from HardwareSwap.Models import Base, engine, Post, PostType, get_or_create
import tqdm

In [ ]:
Base.metadata.create_all(engine)
Session = sessionmaker(bind = engine)

In [ ]:
s = Session()
buy = get_or_create(s, PostType, post_type="buy")
sell = get_or_create(s, PostType, post_type="sell")
trade = get_or_create(s, PostType, post_type="trade")
giveaway = get_or_create(s, PostType, post_type="giveaway")

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport HardwareSwap
%aimport HardwareSwap.DownloadData
%aimport HardwareSwap.DownloadData.download_data
from HardwareSwap.DownloadData import download_data, remove_duplicate_rows
import os
import datetime
import pytz
import pandas as pd

In [ ]:
json_dir = "/home/neil/RandomProjects/hardwareswap/raw_data/"
raw_data = download_data.load_dataframe_from_disk(os.path.join(json_dir, "data_*-*.json"), limit=1000)

print(f"Found {len(raw_data)} posts")


In [ ]:
raw_data = remove_duplicate_rows(raw_data, "id", [idx[0] for idx in s.query(Post.id).all()])

In [ ]:
m = {
    list: "String",
     str: "String",
    dict: "String",
     int: "Integer",
     float: "Float",
     bool: "Boolean",
     np.bool_: "Boolean",
     np.int64: "Integer",
     np.float64: "Float",
    }

needs_json = []
needs_datetime = []
for col in raw_data.columns:
    try:
        col_values = raw_data[col].unique()
        col_types = {type(val) for val in col_values}
    except TypeError:
        col_types = {type([])}
    
    if (len(col_types) > 1) and (type(None) in col_types):
        col_types.remove(type(None))
        
    if (len(col_types) > 1) and (float in col_types):
        col_types.remove(float)
        
    if (list in col_types) or (dict in col_types):
        needs_json.append(col)
    
    sql_type=""
    if len(col_types) == 1 and list(col_types)[0] in m:
        sql_type = m[col_types.pop()]
    
    if "_utc" in col:
        sql_type="DateTime"
        needs_datetime.append(col)
    print(f"{col} = Column({sql_type})")
    if sql_type=="":
        print(f"\t{col_types}")
    
    #if col == "author_flair_background_color":
    #    break
    
print(needs_json)
print(needs_datetime)

In [ ]:
# Determine how many were just deleted
n_author_deleted = s.query(Post).filter(Post.author=="[deleted]").update({"deleted":True})
n_post_deleted =s.query(Post).filter(Post.selftext=="[deleted]").update({"deleted":True})
n_post_removed =s.query(Post).filter(Post.selftext=="[removed]").update({"deleted":True})
s.commit()
print(f"Author Deleted: {n_author_deleted} Post Deleted: {n_post_deleted} Post Removed: {n_post_removed} Total: {s.query(Post.id).filter(Post.deleted==True).count()}")